In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [2]:
# For M
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# For M' and M'' and M*
class Net_sub(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64) # subbed from gaussian
        self.fc2 = nn.Linear(64, 64)  # changed to match dimension
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
  

In [3]:
M = Net()
M.load_state_dict(torch.load('model.pth'))
M.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

### Creating $S_{64\times128}$

In [5]:
shape = (64,128)
S = np.random.normal(0, 1, shape)
S = torch.tensor(S, dtype=torch.float32)

In [ ]:
shape = ()

In [6]:
A_dash_weights = S @ M.fc1.weight.data.clone()
A_dash_bias = S @ M.fc1.bias.data.clone()

In [8]:
print(A_dash_weights.shape, A_dash_bias.shape)

torch.Size([64, 784]) torch.Size([64])


In [ ]:
M_dash = Net_sub()
M_dash.fc1.weight.data = A_dash_weights
M_dash.fc1.bias.data = A_dash_bias
M_dash.fc2.weight.data = M.fc2.weight.data.clone()